In [0]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import spacy
!pip install textacy
import textacy
import pandas as pd
nlp = spacy.load('en')

In [0]:
!unzip -n 'gdrive/My Drive/549_data.zip'

Archive:  gdrive/My Drive/549_data.zip
  inflating: test.csv                
  inflating: train.csv               


In [0]:
train = pd.read_csv('train.csv', sep='\t')
test = pd.read_csv('test.csv', sep='\t')

In [0]:
def ner(text):
    text = nlp(text)
    text_ext = textacy.extract.subject_verb_object_triples(text)
    lte = list(text_ext)
    return lte

In [0]:
train['named_entities'] = train['text'].apply(ner)

In [0]:
found = train[train.apply(lambda x: len(x['named_entities']) != 0, axis=1)]
print(found.shape)

(5462, 3)


In [0]:
'found:', found.shape[0], 'missing:',train.shape[0] - found.shape[0]

((8336, 3), 2874)

Take a look at the extracted parts,notice that not all sentences were parsed succesfully.

In [0]:
print(*found.named_entities, sep='\n')

[(that, 's, girl), (I, 'm giving, Motrin), (She, doesn't like, Tylenol)]
[(I, am going, to need)]
[(I, could pop, Xanax)]
[(I, 'm drinking, pop), (I, have, means), (I, get, work)]
[(head, told, me)]
[(I, took, Tylenol)]
[(Tylenol, is, bestfriend)]
[(I, 've got, card)]
[(him, take, steroids)]
[(@nguerriero19, walgreens, https://www.instagram.com/p/BOp_nXmB1iX/)]
[(I, took, Tylenol), (I, have, head ache), (I, have, ear ache)]
[(I, need, Tylenol)]
[(We, know, Chad), (Jojo, needs, to send), (show, do, this)]
[(I, need, breakfast), (I, need, bottles), (I, need, shit ton)]
[(husband, offered, me), (husband, offered, dick)]
[(I, 'd have, to be)]
[(she, was, take)]
[(it, 's, thank)]
[(strength, works, wonders)]
[(I, need, Tylenol)]
[(it, 's, time)]
[(I, 've had, level headache)]
[(he, is, teething hun)]
[(I, don't know, Guest), (It, 's, Guest)]
[(I, tell, mother)]
[(All, is, Tylenol)]
[(he, 's not, gypsey)]
[(you, take, steroids), (I, 'm judging, outta u)]
[(It, sucks, having)]
[(he, put, me)]

In [0]:
missing = train[train.apply(lambda x: len(x['named_entities']) == 0, axis=1)]

In [0]:
missing.shape

(2874, 3)

In [0]:
missing

,text,label,named_entities
3,Im miserably tired but can't sleep and still w...,1,[]
4,the amount of ibuprofen i take is absolutely h...,2,[]
7,"I'm on codeine, Zpack, Zyrtec, and 40mg of pre...",1,[]
10,spongebob on steroids lmfaoooo,3,[]
12,Tylenol PM for the win,2,[]
...,...,...,...
8318,The #twins are super active this morning! And ...,1,[]
8323,Advil pm time..,2,[]
8324,Is it just me or does Ryan Braun look noticeab...,3,[]
8327,"What steroids do to Levi! In other news, since...",3,[]


In [0]:
def to_sentence(nes):
    acc = [] 
    for ne in nes:
        acc.extend(ne)
        acc.append('.')

    return ' '.join(str(x) for x  in acc)


In [0]:
data = found.named_entities.apply(to_sentence)

In [0]:
data

0       that 's girl . I 'm giving Motrin . She doesn'...
1                                    I am going to need .
2                                     I could pop Xanax .
5         I 'm drinking pop . I have means . I get work .
6                                          head told me .
                              ...                        
8330    I Been Taking Tylenol . Justt Been Taking Tyle...
8331      I 've been taking Tylenol . I haven't had one .
8332                                  It is not IBRUFEN .
8333                               Someone flagged post .
8334    it was pm . dads fault tableconversations . da...
Name: named_entities, Length: 5462, dtype: object

In [0]:
target = train.label.iloc[found.index]
target.shape

(5462,)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np

In [0]:
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
text_clf.fit(data, target.values)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [0]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(data, target.values)

# predicted = text_clf.predict(docs_test)
# np.mean(predicted == twenty_test.target)

In [0]:
test['named_entities'] = test['text'].apply(ner)
test_found = test[test.apply(lambda x: len(x['named_entities']) != 0, axis=1)]
test_data = test_found.named_entities.apply(to_sentence)
test_target = test.label.iloc[test_found.index]

In [0]:

predicted = text_clf.predict(test_data)
np.mean(predicted == test_target)

0.5482520925652388

In [0]:
from sklearn import metrics
print(metrics.classification_report(test_target, predicted))

metrics.confusion_matrix(test_target, predicted)

              precision    recall  f1-score   support

           1       0.75      0.19      0.31       922
           2       0.60      0.47      0.53      1522
           3       0.51      0.83      0.63      1618

    accuracy                           0.55      4062
   macro avg       0.62      0.50      0.49      4062
weighted avg       0.60      0.55      0.52      4062



array([[ 177,  217,  528],
       [  39,  714,  769],
       [  19,  263, 1336]])